In [35]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
from catboost import *
import xgboost
import shap
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_ST.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                       'Operating Temperature (℃)',#0
                      'Operating Pressure (bar)',#1
                      'Flow Rate (mL min-1)',#2
                      'Active Area (cm2)', #3
                      'Ir wt. %',#4
                      'Ru wt.%',#5
                      'O wt. %',#6
                      'C wt. %',#7
                      'Pure_0/Supported_1',#8
                      'I/C in Anode',#9
                      'Pt wt. %',#10
                      'I/C in Cathode',#11  
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#12
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#13
                      'CCM_0/GDE_1',#14    
                      'Membrane Thickness (μm)',#15
                      'EW',#16
                      'Minimum Stability Current Density (A cm-2)',#17
                      'Maximum Stability Current Density (A cm-2)',#18
                      'Fluctuation period (h)',#19
                      'Stability Test Time (h-1)',#20
                      'CL_10'#21
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:21]
raw_power=raw_data.iloc[:,21]
print('ready')

Using matplotlib backend: Qt5Agg
ready


In [36]:
from sklearn.metrics import roc_curve, auc
def auc_curve(y_label, y_pre,algorithm_name):
    y_label = y_label + 1
    y_pre = y_pre + 1
    fpr, tpr, thersholds = roc_curve(y_label, y_pre, pos_label=2)
    roc_auc = auc(fpr, tpr)
    x_line=np.arange(0,1.01,0.01)
    y_line=np.arange(0,1.01,0.01)
    print('auc',roc_auc)
    fig=plt.figure()
    plt.plot(fpr, tpr, 'k--', label='ROC (AUC/area = {0:.2f})'.format(roc_auc), lw=2)
    plt.plot(x_line,y_line,c='red')
    plt.xlim([-0.05, 1.05])  # 设置x、y轴的上下限，以免和边缘重合，更好的观察图像的整体
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')  # 可以使用中文，但需要导入一些库即字体
    plt.title('ROC Curve of %s' %algorithm_name)
    plt.legend(loc="lower right")
    fig.savefig("ROC Curve of %sST10.png" %algorithm_name)
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    y_score=best_model.predict_proba(X_test)
    y_score=y_score[:,1]
    auc_curve(y_test,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
def shap_plot(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
#         shap.dependence_plot('Electronegativity',shap_values,SHAP_INPUT,interaction_index='Number of d electrons')
#         shap.dependence_plot('Number of d electrons',shap_values,SHAP_INPUT,interaction_index='Electronegativity')
#         shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)

In [37]:
seed=1743
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt','rf'],
 'learning_rate':[0.5,0.6,0.7,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1,1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2,1.3,1.4,1.5],
  'n_estimators':[50,100,200,400,None],
 'subsample':[0.3,0.35,0.4,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.6,0.65,0.7],
 'max_depth':[5,7,9,11,-1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],
 'n_estimators':[50,100,200,400,None],
 'max_depth':[5,7,9,11,16],
 'subsample':[0.6,0.65,0.7,0.8,0.85,0.9,0.95,1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
     'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
 'n_estimators':[50,100,200,400],
"boosting_type":["Plain"],
 'max_depth':[5,7,9,11],
 'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]

}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.2,0.25,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],
     'max_depth':[3,5,7,9,11,16],
 'criterion':['friedman_mse','mae','mse'],
 'max_features':['auto','sqrt','log2'],
 'loss':['deviance', 'exponential']
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
  'n_estimators':[10,50,100,200,400],
     'max_depth':[3,5,7,9,11,None],
     'criterion':['gini','entropy'],
     'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
    'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
    'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
      'learning_rate':[0.5,0.6,0.7,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1,1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2,1.3,1.4,1.5],
    'n_estimators':[30,40,50,100,200]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

In [21]:
##########LightGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt'],
 'learning_rate':[0.054],
  'n_estimators':[100],
 'subsample':[0.3],
 'max_depth':[7],
 'reg_alpha':[0],
 'reg_lambda':[0]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

start
Best Classifier: {'boosting_type': 'gbdt', 'learning_rate': 0.054, 'max_depth': 7, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 0.3} Best Score: 0.7942857142857143
auc 1.0
              precision    recall  f1-score   support

           0       0.83      0.96      0.89        56
           1       0.75      0.35      0.48        17

    accuracy                           0.82        73
   macro avg       0.79      0.66      0.69        73
weighted avg       0.81      0.82      0.80        73

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         1

    accuracy                           0.93        14
   macro avg       0.46      0.50      0.48        14
weighted avg       0.86      0.93      0.89        14

0.9285714285714286
LightGBM
[ 0  0  2 34  0  0 31  0  0 28 10 37 31  7  0  0  5  8  0  0  7]


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [39]:
shap_plot(model_LightGBMClassifier,param_light,'LightGBM')

LightGBM
[0.         0.         0.02641713 0.62882005 0.         0.
 0.47903317 0.         0.         0.24485615 0.15043259 0.4432542
 0.33249305 0.04120127 0.         0.         0.07774477 0.07740887
 0.         0.         0.07981399]


In [22]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbose=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.02],
'n_estimators':[100],
'max_depth':[5],
'subsample':[0.78],
'reg_alpha':[0],
'reg_lambda':[0]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

start
Best Classifier: {'booster': 'gbtree', 'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 0.78} Best Score: 0.7504761904761905
auc 0.9230769230769231
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        17

    accuracy                           1.00        73
   macro avg       1.00      1.00      1.00        73
weighted avg       1.00      1.00      1.00        73

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       0.50      1.00      0.67         1

    accuracy                           0.93        14
   macro avg       0.75      0.96      0.81        14
weighted avg       0.96      0.93      0.94        14

0.9285714285714286
XGBoost
[0.015099   0.00550622 0.0211123  0.03562072 0.04135456 0.04833227
 0.07793649 0.09098978 0.00999677 0.03941

In [40]:
shap_plot(model_XGBClassifier,param_xg,'XGBoost')

XGBoost
[0.00350054 0.00035113 0.01611557 0.01612993 0.01403402 0.00779379
 0.06361223 0.00023746 0.00044794 0.02657596 0.02514209 0.05352681
 0.04301618 0.03134734 0.         0.0007706  0.00035712 0.05266498
 0.01685824 0.00700787 0.03545845]


In [23]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
'learning_rate':[1],
'n_estimators':[50],
'max_depth':[7],
'subsample':[1],
'reg_lambda':[0.0001]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

start
Best Classifier: {'learning_rate': 1, 'max_depth': 7, 'n_estimators': 50, 'reg_lambda': 0.0001, 'subsample': 1} Best Score: 0.7657142857142858
auc 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        17

    accuracy                           1.00        73
   macro avg       1.00      1.00      1.00        73
weighted avg       1.00      1.00      1.00        73

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         1

    accuracy                           1.00        14
   macro avg       1.00      1.00      1.00        14
weighted avg       1.00      1.00      1.00        14

1.0
CatBoost
[2.46517358e+00 0.00000000e+00 6.22846237e+00 1.67682543e+01
 0.00000000e+00 3.85204677e+00 1.42957063e+01 1.06500559e-03
 1.07418257e+00 1.18697872e+00 3.32506628e+00 1.213995

In [41]:
shap_plot(model_CatClassifier,param_cat,'CatBoost')

CatBoost
[0.67360713 0.         1.12314746 2.39867244 0.         1.09075208
 3.13296142 0.03150898 0.43414978 0.62885431 0.93720839 0.03219012
 2.86879428 1.60049418 0.         1.7466093  0.32951494 0.13871926
 2.9513547  0.55888888 0.33072011]


In [24]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.005],
'criterion':['mae'],
'max_features':['auto'],
'loss':['deviance'],
'max_depth':[11]
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

start
Best Classifier: {'criterion': 'mae', 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 11, 'max_features': 'auto'} Best Score: 0.74
auc 0.9615384615384616
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        56
           1       1.00      0.82      0.90        17

    accuracy                           0.96        73
   macro avg       0.97      0.91      0.94        73
weighted avg       0.96      0.96      0.96        73

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       0.50      1.00      0.67         1

    accuracy                           0.93        14
   macro avg       0.75      0.96      0.81        14
weighted avg       0.96      0.93      0.94        14

0.9285714285714286
GradientBoost
[1.89251730e-04 7.12511523e-04 7.44723710e-03 6.62438508e-03
 5.61981402e-02 6.52446351e-02 1.48361507e-01 2.36514695e-05
 8.46385222e-04 1.1861

In [42]:
shap_plot(model_GradientBoostingClassifier,param_GB,'GradientBoost')

GradientBoost
[4.74003785e-03 4.89831543e-04 1.33865480e-01 3.73971249e-03
 2.79997327e-02 9.97841295e-02 7.98210703e-02 9.57729766e-06
 2.34247767e-03 1.23502968e-02 1.36801899e-01 8.95716599e-02
 6.67636378e-02 8.69186258e-03 0.00000000e+00 2.68381700e-03
 4.15197574e-03 5.59495906e-02 5.47340633e-02 1.89053181e-02
 2.63767327e-01]


In [25]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[10],
'max_depth':[7],
'criterion':['gini'],
'max_features':['auto']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')

start
Best Classifier: {'criterion': 'gini', 'max_depth': 7, 'max_features': 'auto', 'n_estimators': 10} Best Score: 0.7942857142857143
auc 0.923076923076923
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        17

    accuracy                           1.00        73
   macro avg       1.00      1.00      1.00        73
weighted avg       1.00      1.00      1.00        73

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       0.50      1.00      0.67         1

    accuracy                           0.93        14
   macro avg       0.75      0.96      0.81        14
weighted avg       0.96      0.93      0.94        14

0.9285714285714286
Random Forest
[0.         0.         0.05599971 0.00715128 0.02414409 0.04257142
 0.07176231 0.         0.0002773  0.13344213 0.03486646 0.12106733
 0.12205028 0.0355459

In [43]:
shap_plot(model_RandomForestClassifier,param_RF,'Random Forest')

Random Forest
[0.         0.         0.03818432 0.01493768 0.01192432 0.01012654
 0.04816476 0.         0.00031585 0.03414226 0.02684217 0.04199606
 0.03236734 0.03880678 0.         0.00427465 0.01287092 0.02885386
 0.01683257 0.00793852 0.06029976]


In [44]:
###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
'max_depth':[6],
'criterion' : ['entropy'],
'splitter' : [ "random"],
'max_features':['auto']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')

start
Best Classifier: {'criterion': 'entropy', 'max_depth': 6, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.74
auc 0.34615384615384615


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


              precision    recall  f1-score   support

           0       0.88      1.00      0.93        56
           1       1.00      0.53      0.69        17

    accuracy                           0.89        73
   macro avg       0.94      0.76      0.81        73
weighted avg       0.90      0.89      0.88        73

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         1

    accuracy                           0.93        14
   macro avg       0.46      0.50      0.48        14
weighted avg       0.86      0.93      0.89        14

0.9285714285714286
Extra Tree
[0.         0.         0.         0.         0.         0.
 0.13992044 0.         0.0774162  0.         0.05116041 0.0665216
 0.07914869 0.         0.         0.07027116 0.03813809 0.12942396
 0.0951483  0.         0.25285116]


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [45]:
shap_plot(model_ExtraTreeClassifier,param_ET,'Extra Tree')

Extra Tree
[0.         0.         0.         0.         0.         0.
 0.05873813 0.         0.03910536 0.         0.01787995 0.02234949
 0.02710471 0.         0.         0.0060554  0.03152919 0.08133168
 0.05748562 0.         0.05635605]


In [46]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
'max_depth':[6],
'criterion' : ['entropy'],
'splitter' : [ "random"],
'max_features':['auto']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')

start
Best Classifier: {'criterion': 'entropy', 'max_depth': 6, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.74
auc 0.34615384615384615


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


              precision    recall  f1-score   support

           0       0.88      1.00      0.93        56
           1       1.00      0.53      0.69        17

    accuracy                           0.89        73
   macro avg       0.94      0.76      0.81        73
weighted avg       0.90      0.89      0.88        73

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         1

    accuracy                           0.93        14
   macro avg       0.46      0.50      0.48        14
weighted avg       0.86      0.93      0.89        14

0.9285714285714286
Decision Tree
[0.         0.         0.         0.         0.         0.
 0.13992044 0.         0.0774162  0.         0.05116041 0.0665216
 0.07914869 0.         0.         0.07027116 0.03813809 0.12942396
 0.0951483  0.         0.25285116]


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [47]:
shap_plot(model_DecisionTreeClassifier,param_DT,'Decision Tree')

Decision Tree
[0.         0.         0.         0.         0.         0.
 0.05873813 0.         0.03910536 0.         0.01787995 0.02234949
 0.02710471 0.         0.         0.0060554  0.03152919 0.08133168
 0.05748562 0.         0.05635605]


In [29]:
###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
'n_estimators':[200],
'learning_rate':[0.007]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

start
Best Classifier: {'learning_rate': 0.007, 'n_estimators': 200} Best Score: 0.7809523809523808
auc 0.9230769230769231


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


              precision    recall  f1-score   support

           0       0.78      1.00      0.88        56
           1       1.00      0.06      0.11        17

    accuracy                           0.78        73
   macro avg       0.89      0.53      0.49        73
weighted avg       0.83      0.78      0.70        73

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.00      0.00      0.00         1

    accuracy                           0.93        14
   macro avg       0.46      0.50      0.48        14
weighted avg       0.86      0.93      0.89        14

0.9285714285714286
AdaBoost
[0.    0.    0.    0.045 0.    0.    0.115 0.    0.    0.02  0.    0.
 0.09  0.09  0.    0.    0.    0.145 0.18  0.    0.315]


In [48]:
shap_plot(model_AdaBoostClassifier,param_Ada,'AdaBoost')

AdaBoost
[0.         0.         0.         0.00666688 0.         0.
 0.00988334 0.         0.         0.00238059 0.         0.
 0.0110217  0.00857503 0.         0.         0.         0.0145942
 0.01892345 0.         0.02228076]
